# Phase 3: Setting up the BEAST xmls

<details>
    <summary>Click To See A Decription of Parameters</summary>
        <pre>
            <code>
save_dir: str  
    Path to directory for saving outputs in.

template_xml_path: str
    Path to template BEAST xml.

use_initial_tree:  bool, default True
    Is there an inital tree to be used. If not the initial tree will not be used in generating a BEAST 2 xml
    and will BEAST 2 generate its own.

rt_dims: int, optional
    Number of Rt dimensions (time periods over which Rt is estimated).

rt_change_dates: : list, tuple, pd.Series or pd.DatetimeIndex of datetimes
    Internal partitions of Rt estimation periods.

sampling_prop_dims: int, optional
    Number of sampling proportion dimensions (time periods over which sampling proportion is estimated).

sampling_prop_change_dates: : list, tuple, pd.Series or pd.DatetimeIndex of dates
    Internal partitions of sampling proportion estimation periods.

metadata_path: str
       Path to csv or tsv containing metadata.

collection_date_field: str, default='date'
    Name of field in metadata_db containing collection dates of sequences. Should be format YYYY-MM-DD.

sample_id_field: str, default 'strain'
    Name of field in metadata_db containing ids corresponding to those used in fasta_path.

initial_tree_path: str
    Path to initial_tree. Should .nwk file.

fasta_path: str
    Path to fasta file containing sequences.

origin_start_addition float
    This + initial temporal tree height is used as starting value of origin.
    We recommend using an estimate of infection period for the pathogen being studied. **Value should be in years.**
    Origin prio will be unform:
        Lower value: time in years from oldest to youngest sequence in fasta_path
        Start value: origin_start_addition + initial temporal tree height
        Upper value:  initial temporal tree height + origin_upper_addition.

origin_upper_addition: float/int
    This + initial temporal tree height is used as upper value of origin prior. **Value should be in years.**
    Origin prio will be unform:
        Lower value: time in years from oldest to youngest sequence in fasta_path
        Start value: origin_start_addition + initial temporal tree height
        Upper value:  initial temporal tree height + origin_upper_addition.

origin_prior: dict {'lower': float, 'upper': float, 'start': float}, optional
       Details of the origin prior. assumed to be uniformly distributed.

log_file_basename: str, optional
    If provided .tree, .log and .state files from running BEAST 2 will have this name prefixed by 'run-with-seed-{seed}-',
    number being that of the chain.

chain_length: int
    Number of chains to use for BEAST runs.

trace_log_every: int
    How often to save a log file during BEAST runs.

tree_log_every: int
    How often to save a tree file during BEAST runs.

screen_log_every: int
    How often to output to screen during BEAST runs.

store_state_every: int 
    How often to store MCMC state during BEAST runs.
  </code>
</pre>


In [ ]:
save_dir = None
template_xml_path = None
fasta_path = None
use_initial_tree = True
initial_tree_path = None
metadata_path = None
rt_dims = None
rt_partitions = None
sampling_prop_dims=None
sampling_prop_partitions=None
zero_sampling_before_first_sample=True
collection_date_field = 'date'
sample_id_field='strain'
origin_upper_addition = None
origin_prior = None
origin_start_addition = None
log_file_basename=None
chain_length = None
trace_log_every = None
tree_log_every = None
screen_log_every = None
store_state_every = None

Import packages. 

In [ ]:
import os
from beast_pype.beast_xml_gen import gen_bdsky_serial_xml
import pandas as pd

### Search for files in save_dir if not provided

In [ ]:
if use_initial_tree:
    if initial_tree_path is None:
        initial_tree_path = f'{save_dir}/initial_trees/down_sampled_time.nwk'
        if not os.path.exists(initial_tree_path):
            initial_tree_path = f'{save_dir}/initial_trees/full_time.nwk'
            if not os.path.exists(initial_tree_path):
                initial_tree_path = f'{save_dir}/initial_trees/iqtree.nwk'
                if not os.path.exists(initial_tree_path):
                    raise FileNotFoundError(f'Initial tree file not found. initial_tree_path has not been provided and none of the files down_sampled_time.nwk, full_time.nwk or iqtree.treefile can not be found in save_dir ({save_dir}).')

if metadata_path is None:
    metadata_path = f'{save_dir}/down_sampled_metadata.csv'
    if not os.path.exists(metadata_path):
        metadata_path = f'{save_dir}/metadata.csv'
    if not os.path.exists(metadata_path):
        raise FileNotFoundError(f'Metadata file not found. \n' +
                                f'A metadata_path has not been provided and neither the file down_sampled_metadata.csv or metadata.csv can be found in save_dir ({save_dir}).')

if fasta_path is None:
    fasta_path = f'{save_dir}/down_sampled_sequences.fasta'
    if not os.path.exists(fasta_path):
        fasta_path = f'{save_dir}/sequences.fasta'
    if not os.path.exists(fasta_path):
        raise FileNotFoundError(f'Fasta file not found.  \n' +
                                f'fasta_path has not been provided and neither the file down_sampled_sequences.fasta or sequences.fasta can be found in save_dir ({save_dir}).')


if not use_initial_tree and initial_tree_path is not None:
    raise AssertionError('use_initial_tree is False but you have provided an initial_tree_path?')


## Check the Ratio of Sequences to Dimensions is Reasonable

In [ ]:
unsafe_sequence_to_dims = 5
number_of_sequences  = len(pd.read_csv(metadata_path))
if rt_partitions is not None:
    seqs_to_rt_dims = number_of_sequences / (len(rt_partitions) + 1)
    if seqs_to_rt_dims <= unsafe_sequence_to_dims:
        raise ValueError(
            f'The ratio of fasta_path to Rt dimensions is too low ({str(seqs_to_rt_dims)}<={str(unsafe_sequence_to_dims)}.\n' +
            'Suggest you alter rt_partitions to lower the number of partions it produces.')
if sampling_prop_partitions is not None:
    seqs_to_sample_prop_dims = number_of_sequences / (len(sampling_prop_partitions) + 1)
    if seqs_to_sample_prop_dims <= unsafe_sequence_to_dims:
        raise ValueError(
            f'The ratio of fasta_path to Sampling Proportion dimensions is too low ({str(seqs_to_sample_prop_dims)}<={str(unsafe_sequence_to_dims)}.\n' +
            'Suggest you alter rt_partitions to lower the number of partions it produces.')
if rt_dims is not None:
    seqs_to_rt_dims = number_of_sequences / rt_dims
    if seqs_to_rt_dims <= unsafe_sequence_to_dims:
        raise ValueError(
            f'The ratio of fasta_path to Rt dimensions is too low ({str(seqs_to_rt_dims)}<={str(unsafe_sequence_to_dims)}.\n' +
            'Suggest you lower rt_dims.')
if sampling_prop_dims is not None:
    seqs_to_sampling_prop_dims = number_of_sequences / sampling_prop_dims
    if seqs_to_sampling_prop_dims <= unsafe_sequence_to_dims:
        raise ValueError(
            f'The ratio of fasta_path to Rt dimensions is too low ({str(seqs_to_sampling_prop_dims)}<={str(unsafe_sequence_to_dims)}.\n' +
            'Suggest you lower sampling_prop_dims.')

## Generating the BEAST2 xmls.

In [ ]:
gen_bdsky_serial_xml(
    template_path=template_xml_path,
    sequences_path=fasta_path,
    metadata_path=metadata_path,
    initial_tree_path=initial_tree_path,
    origin_prior=origin_prior,
    collection_date_field=collection_date_field,
    sample_id_field=sample_id_field,
    origin_upper_height_addition=origin_upper_addition,
    origin_start_addition=origin_start_addition,
    output_path=f"{save_dir}/beast.xml",
    rt_dims=rt_dims,
    rt_change_dates=rt_partitions,
    sampling_prop_dims=sampling_prop_dims,
    sampling_prop_change_dates=sampling_prop_partitions,
    #  For when we can get this working in beast2xml see https://github.com/acorg/beast2-xml/issues/21#issuecomment-3553733538.
    # zero_sampling_before_first_sample=zero_sampling_before_first_sample,
    log_file_basename=log_file_basename,
    chain_length=chain_length,
    trace_log_every=trace_log_every,
    tree_log_every=tree_log_every,
    screen_log_every=screen_log_every,
    store_state_every=store_state_every
)